In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install transformers

In [3]:
from transformers import BertTokenizerFast, GPT2LMHeadModel

In [12]:
gens=["<인디>","<힙합>"] 
# gens=["<힙합>"]

tokenizer_gpt3 = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2", 
                                                   bos_token='<|startoftext|>', # begin of string [bos] token
                                                   eos_token='<|endoftext|>',  # end of string  [eos] token
                                                   pad_token='<|pad|>', # padding token
                                                   additional_special_tokens=gens)
 
tokenizer_gpt3.add_tokens("<br>")

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [5]:
# 구글 드라이브에서 모델, 토크나이저 불러오기

ind_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/KUBIG/NLP/KUBIG Project/ind_model_save')
hip_model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/KUBIG/NLP/KUBIG Project/hip_model_save')

# gpt3-kor-small_based_on_gpt2
# ind_tokenizer = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/KUBIG/NLP/KUBIG Project/ind_model_save') 
# hip_tokenizer = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/KUBIG/NLP/KUBIG Project/hip_model_save') 

# ind_input_ids = ind_tokenizer.encode("text to tokenize")[1:]  # remove cls token
# input_ids_hip = hip_tokenizer.encode("text to tokenize")[1:]  # remove cls token

In [6]:
import torch
import random
import numpy as np

In [7]:
device = torch.device("cuda")
ind_model.cuda()
hip_model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

ind_model = ind_model.to(device)
hip_model=hip_model.to(device)

----------------------

In [79]:
begin_of_string="오늘밤에"

In [80]:
outputs=tokenizer_gpt3.encode(begin_of_string) 

In [81]:
outputs

[2, 14098, 31835, 3]

In [82]:
for i in outputs[1:-1]:
  print(tokenizer_gpt3.convert_ids_to_tokens(i))

오늘
##밤에


In [83]:
start_id=outputs[1]
keyword=tokenizer_gpt3.convert_ids_to_tokens(start_id)
print('keyword :',keyword)
begin_length=len(keyword)
print(begin_length)

keyword : 오늘
2


# Indie Lyrics Generator

In [84]:
ind_model.eval()
# generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
# generated = generated.to(device)

# print(generated)

sample_outputs =ind_model.generate(                    
                                bos_token_id=start_id,
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3,
                                repetition_penalty=1.2
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {} {}\n\n".format(i, begin_of_string,tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)[begin_length+1:]))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


0: 오늘밤에 오 지금쯤 어디 있을까 난 너를 찾고 있었지 너의 눈동자 푸른 빛깔의 작은 달을 찾아 헤매이고 있겠지 그 까만 밤 속에 나의 모습이 조금 두려워 떨고 있는 내게 손짓하던 미소 보이겠지 내 곁으로 와줄 수 있어 나는 지금 행복해 이대로 널 잃어가도 괜찮으니까 어느새 넌 나를 잊어가고 있겠지 내가 그리워했던 날들이 희미하게 다가오는데 그리움도 시간이 흘러 감에 따라 점점 흐려지고 있나 봐 어느새 니가 그립네 아 눈부시게 밝아오던 그날이 떠올라 다시 한 번 나 여기 남겨볼까 이 늦은 밤에 혼자 어딜 가고 있는지 넌 모를 텐데 조금씩 지워져도 괜찮은 걸까 어느덧 넌 나를 찾으면서 점점 흐려져 가겠지 그리움이 지나고 나면 모두 잊혀질까 그렇게 기다리다 또 눈물이 흐르면 어떡하니 떠나가던 눈빛들 이젠 다 사라져가도 상관없어 함께였던 순간은 소중하니까


1: 오늘밤에 참 이상하죠 당신도 내 생각 할까요 당신도 내 생각 할까요 난 늘상 그래요 매일 웃네요 당신은 언제나 사랑인가요 나를 떠나간 당신의 마음도 그리운걸 이제 또 다시 웃음 지으며 하루하루 당신을 생각해 그런가요 항상 그자리에 있죠 언제나 그렇게 웃죠 당신은 사랑인가요 나는 정말 사랑하는데 이젠 울어도 되는데 왜 이렇게 아픈가요 내게 돌을 던지세요 이미 돌이킬수 없죠 내가 너무 이기적이란 걸 나도 알고 있는데 당신은 나약했죠 날 버리고 간 당신의 가슴도 그리웠는걸 이젠 모두 다 잊었나요 항상 내 옆에 있죠 언제나 그렇게 웃죠 당신은 사랑인가요 사랑은 사랑이에요 이제는 사랑을 해야해요


2: 오늘밤에 눈이 부시던 맑은 날 너는 다시 내게로 내려와 하얗고 보드라운 너의 손을 잡아주네 눈부시게 반짝이던 세상도 차갑게 식어가네 나는 다시 한 번 더 그대를 보고 싶었어 그댄 나의 맘을 알까요 눈을 감은 두 사람의 목소리와 귓가에 들렸던 달콤한 멜로디 기억나니 이 세상 아름다운 사람 너무 많아 그대여 그대 나와 함께 가요 아름다워요




# Hiphop Lyrics Generator

In [85]:
hip_model.eval()
# generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
# generated = generated.to(device)

# print(generated)

sample_outputs =hip_model.generate(            
                                bos_token_id=start_id,
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3,
                                repetition_penalty=1.2
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {} {}\n\n".format(i, begin_of_string,tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)[begin_length+1:]))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


0: 오늘밤에 우린 한 치 앞도 안 보고 달려 더 높이 가야 해 너네들 손 위에 다 갖다 놔 난 너무 높아 더 높게 가야해 내 몸 위에는 전부 다 네게 맡겨 놓고 떠나 우리끼리 놀 때는 어디든 네가 있으니 넌 걱정하지 말어 내가 제일 잘하니까 너도 열심히 살아야겠지 baby girl i need one you know there it aint no time for me dont even stop yeah better go back 우리만의 세상으로 들어와 im in love with us if youre so shorty to get a good luck booming on me and lets have taste my thang just tellem out its all alone we can drivin at this world we will comes up sober come her right now baby girl i need one you know there it aint no time for me dont even stop yeah better go back 우리만의 세상으로 들어와 im in love with us if youre so shorty to get a good luck booming on me and lets have taste my thang just tellem out its all alone we can drivin at this world we will comes up


1: 오늘밤에 intro i put it back in ma clouds and we boots for a big guts oh yeah they sicken drunk on my taked up i need that veteran left on your head on em from bruce wayne jackson shawty im feeling like sushi 난 그냥 널 원해 i know you said 네 눈동자는 빛나 니 맘은 얼어붙을 것만 같